In [ ]:
!pip install patchify

In [ ]:
import os
import cv2 as cv
import numpy as np
from patchify import patchify
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical
from matplotlib import pyplot as plt
from google.colab.patches import cv2_imshow

In [ ]:
minmaxscaler = MinMaxScaler()

In [ ]:
!ls -lah "/content/drive/MyDrive/Semantic segmentation dataset"

total 33K
-rw------- 1 root root  548 Aug 24 05:16  classes.json
drwx------ 4 root root 4.0K Aug 25 07:41 'Tile 1'
drwx------ 4 root root 4.0K Aug 25 07:41 'Tile 2'
drwx------ 4 root root 4.0K Aug 25 07:41 'Tile 3'
drwx------ 4 root root 4.0K Aug 25 07:41 'Tile 4'
drwx------ 4 root root 4.0K Aug 25 07:41 'Tile 5'
drwx------ 4 root root 4.0K Aug 25 07:41 'Tile 6'
drwx------ 4 root root 4.0K Aug 25 07:41 'Tile 7'
drwx------ 4 root root 4.0K Aug 25 07:41 'Tile 8'


In [ ]:
dataset_root_folder = "/content/drive/MyDrive/"

In [ ]:
dataset_name = "Semantic segmentation dataset"

In [ ]:
# for path, sub_dirs, files in os.walk(os.path.join(dataset_root_folder, dataset_name)):
#   dir_name = path.split(os.path.sep)[-1]
#   if dir_name == "masks":
#     images = os.listdir(path)
#     for i, image_name in enumerate(images):
#       if image_name.endswith('.png'):
#         a = True


In [ ]:
image_dataset = []
mask_dataset = []
image_patch_size = 256

In [ ]:
for image_type in ["images", "masks"]:
  if image_type == "images":
    image_extension = "jpg"
  elif image_type == "masks":
    image_extension = "png"
  for tile_number in range(1,9):
    for image_number in range(1,20):
      image = cv.imread(f'{dataset_root_folder}/{dataset_name}/Tile {tile_number}/{image_type}/image_part_00{image_number}.{image_extension}', 1)
      if image is not None:
        if image_type == "masks":
          image = cv.cvtColor(image, cv.COLOR_BGR2RGB)
        patch_height = (image.shape[0]//image_patch_size)*image_patch_size
        patch_width = (image.shape[1]//image_patch_size)*image_patch_size
        image = image[:patch_height, :patch_width]
        patched_images = patchify(image, (image_patch_size, image_patch_size, 3), step=image_patch_size)
        for i in range(patched_images.shape[0]):
          for j in range(patched_images.shape[1]):
            individual_patch = patched_images[i, j, :, :]
            if image_type == "images":
              individual_patch = minmaxscaler.fit_transform(individual_patch.reshape(-1, individual_patch.shape[-1])).reshape(individual_patch.shape)
              individual_patched_image = individual_patch[0]
              image_dataset.append(individual_patched_image)
            elif image_type == "masks":
              individual_patched_mask = individual_patch[0]
              mask_dataset.append(individual_patched_mask)

In [ ]:
plt.figure(figsize=(10,10))
plt.subplot(121)
plt.imshow(image_dataset[0])
plt.subplot(122)
plt.imshow(mask_dataset[0])

In [ ]:
class_building = np.array([int("3c1098"[i:i+2], 16) for i in (0, 2, 4)])
class_land = np.array([int("8429f6"[i:i+2], 16) for i in (0, 2, 4)])
class_road = np.array([int("6ec1e4"[i:i+2], 16) for i in (0, 2, 4)])
class_vegetation = np.array([int("fedd3a"[i:i+2], 16) for i in (0, 2, 4)])
class_water = np.array([int("e2a929"[i:i+2], 16) for i in (0, 2, 4)])
class_unlabeled = np.array([int("9b9b9b"[i:i+2], 16) for i in (0, 2, 4)])

In [ ]:
labels = []
label = individual_patched_mask

In [ ]:
def rgb_to_label(label):
  label_segment = np.zeros(label.shape, dtype=np.uint8)
  label_segment[np.all(label == class_water, axis=-1)] = 0
  label_segment[np.all(label == class_land, axis=-1)] = 1
  label_segment[np.all(label == class_road, axis=-1)] = 2
  label_segment[np.all(label == class_building, axis=-1)] = 3
  label_segment[np.all(label == class_vegetation, axis=-1)] = 4
  label_segment[np.all(label == class_unlabeled, axis=-1)] = 5
  label_segment = label_segment[:, :, 0]

  return label_segment

In [ ]:
for i in range(len(mask_dataset)):
  label = rgb_to_label(mask_dataset[i])
  labels.append(label)

In [ ]:
labels = np.array(labels)

In [ ]:
labels = np.expand_dims(labels, axis=3)

In [ ]:
plt.figure(figsize=(10,10))
plt.subplot(121)
plt.imshow(image_dataset[10])
plt.subplot(122)
plt.imshow(labels[10][:,:,0])

In [ ]:
total_classes = len(np.unique(labels))

In [ ]:
master_training_dataset = np.array(image_dataset)

In [ ]:
labels_categorical_dataset = to_categorical(labels, num_classes = total_classes)

In [ ]:
import pickle

In [ ]:
# with open('master_training_dataset.pkl', 'wb') as f:
#     pickle.dump(master_training_dataset, f)
with open('labels_categorical_dataset.pkl', 'wb') as f:
    pickle.dump(labels_categorical_dataset, f)

In [ ]:
with open('master_training_dataset.pkl', 'rb') as f:
    master_training_dataset_loaded = pickle.load(f)
    print(master_training_dataset_loaded)

[[[[0.78823529 0.76470588 0.74509804]
   [0.88235294 0.85882353 0.83921569]
   [0.94509804 0.90980392 0.89411765]
   ...
   [0.89411765 0.85882353 0.82352941]
   [0.85882353 0.83137255 0.79607843]
   [0.87843137 0.85098039 0.81568627]]

  [[0.88627451 0.8627451  0.84313725]
   [0.99607843 0.97254902 0.95294118]
   [1.         1.         0.98431373]
   ...
   [0.93333333 0.89803922 0.8627451 ]
   [0.92941176 0.90196078 0.86666667]
   [0.95294118 0.91764706 0.88235294]]

  [[0.97254902 0.94509804 0.93333333]
   [1.         0.98823529 0.97647059]
   [1.         1.         0.98431373]
   ...
   [0.93333333 0.89411765 0.86666667]
   [0.92156863 0.89411765 0.85882353]
   [0.90196078 0.86666667 0.83137255]]

  ...

  [[0.88235294 0.8627451  0.86666667]
   [0.81960784 0.80392157 0.8       ]
   [0.7372549  0.73333333 0.71764706]
   ...
   [0.7254902  0.68235294 0.72156863]
   [0.85490196 0.82352941 0.86666667]
   [0.67843137 0.65882353 0.69411765]]

  [[0.76862745 0.75294118 0.74901961]
   [0.7

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(master_training_dataset, labels_categorical_dataset, test_size=0.20, random_state=100)

In [ ]:
image_height = x_train.shape[1]
image_width = x_train.shape[2]
image_channels = x_train.shape[3]
total_classes = y_train.shape[3]

Part 2

In [ ]:
!pip install segmentation-models==1.0.1

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.7/50.7 kB 1.8 MB/s eta 0:00:00


In [ ]:
from keras.models import Model
from keras.layers import Input, Conv2D, MaxPooling2D, UpSampling2D, concatenate, Conv2DTranspose, BatchNormalization, Dropout, Lambda
from keras import backend as K

In [ ]:
def jaccard_coef(y_true, y_pred):
  y_true_flatten = K.flatten(y_true)
  y_pred_flatten = K.flatten(y_pred)
  intersection = K.sum(y_true_flatten * y_pred_flatten)
  final_coeff_value = (intersection + 1.0) / (K.sum(y_true_flatten) + K.sum(y_pred_flatten) - intersection + 1.0)
  return final_coeff_value

In [ ]:
def multi_unet_model(n_classes=5, image_height=256, image_width=256, image_channels=3):
  inputs = Input((image_height, image_width, image_channels))
  source_input = inputs

  c1 = Conv2D(16, (3,3), activation='relu', kernel_initializer='he_normal', padding='same')(source_input)
  c1 = Dropout(0.2)(c1)
  c1 = Conv2D(16, (3,3), activation='relu', kernel_initializer='he_normal', padding='same')(c1)
  p1 = MaxPooling2D((2,2))(c1)

  c2 = Conv2D(32, (3,3), activation='relu', kernel_initializer='he_normal', padding='same')(p1)
  c2 = Dropout(0.2)(c2)
  c2 = Conv2D(32, (3,3), activation='relu', kernel_initializer='he_normal', padding='same')(c2)
  p2 = MaxPooling2D((2,2))(c2)

  c3 = Conv2D(64, (3,3), activation='relu', kernel_initializer='he_normal', padding='same')(p2)
  c3 = Dropout(0.2)(c3)
  c3 = Conv2D(64, (3,3), activation='relu', kernel_initializer='he_normal', padding='same')(c3)
  p3 = MaxPooling2D((2,2))(c3)

  c4 = Conv2D(128, (3,3), activation='relu', kernel_initializer='he_normal', padding='same')(p3)
  c4 = Dropout(0.2)(c4)
  c4 = Conv2D(128, (3,3), activation='relu', kernel_initializer='he_normal', padding='same')(c4)
  p4 = MaxPooling2D((2,2))(c4)

  c5 = Conv2D(256, (3,3), activation='relu', kernel_initializer='he_normal', padding='same')(p4)
  c5 = Dropout(0.3)(c5)
  c5 = Conv2D(256, (3,3), activation='relu', kernel_initializer='he_normal', padding='same')(c5)

  u6 = Conv2DTranspose(128, (2,2), strides=(2,2), padding='same')(c5)
  u6 = concatenate([u6, c4])
  c6 = Conv2D(128, (3,3), activation='relu', kernel_initializer='he_normal', padding='same')(u6)
  c6 = Dropout(0.2)(c6)
  c6 = Conv2D(128, (3,3), activation='relu', kernel_initializer='he_normal', padding='same')(c6)

  u7 = Conv2DTranspose(64, (2,2), strides=(2,2), padding='same')(c6)
  u7 = 70([u7, c3])
  c7 = Conv2D(64, (3,3), activation='relu', kernel_initializer='he_normal', padding='same')(u7)
  c7 = Dropout(0.2)(c7)
  c7 = Conv2D(64, (3,3), activation='relu', kernel_initializer='he_normal', padding='same')(c7)

  u8 = Conv2DTranspose(32, (2,2), strides=(2,2), padding='same')(c7)
  u8 = concatenate([u8, c2])
  c8 = Conv2D(32, (3,3), activation='relu', kernel_initializer='he_normal', padding='same')(u8)
  c8 = Dropout(0.2)(c8)
  c8 = Conv2D(32, (3,3), activation='relu', kernel_initializer='he_normal', padding='same')(c8)

  u9 = Conv2DTranspose(16, (2,2), strides=(2,2), padding='same')(c8)
  u9 = concatenate([u9, c1], axis=3)
  c9 = Conv2D(16, (3,3), activation='relu', kernel_initializer='he_normal', padding='same')(u9)
  c9 = Dropout(0.2)(c9)
  c9 = Conv2D(16, (3,3), activation='relu', kernel_initializer='he_normal', padding='same')(c9)

  outputs = Conv2D(n_classes, (1,1), activation='softmax')(c9)
  model = Model(inputs=[inputs], outputs=[outputs])
  return model

In [ ]:
metrics = ['accuracy', jaccard_coef]

In [ ]:
def get_deep_learning_model():
  return multi_unet_model(n_classes=total_classes, image_height=image_height, image_width=image_width, image_channels=image_channels)

In [ ]:
model = get_deep_learning_model()

In [ ]:
## model configuration
# model.get_config()

Generating Loss Function

In [ ]:
%env SM_FRAMEWORK=tf.keras

env: SM_FRAMEWORK=tf.keras


In [ ]:
import segmentation_models as sm

Segmentation Models: using `tf.keras` framework.


In [ ]:
weights = [0.1666, 0.1666, 0.1666, 0.1666, 0.1666, 0.1666]

In [ ]:
dice_loss = sm.losses.DiceLoss(class_weights=weights)
focal_loss = sm.losses.CategoricalFocalLoss()
total_loss = dice_loss + (1 * focal_loss)

Modal Compilation

In [ ]:
import tensorflow as tf

In [ ]:
tf.keras.backend.clear_session()

In [ ]:
model.compile(optimizer='adam', loss=total_loss, metrics=metrics)

In [ ]:
## Modal summary, similar to netron visualisation
# model.summary()

In [ ]:
model_history = model.fit(x_train, y_train, batch_size=16, verbose=1, epochs=200, validation_data=(x_test, y_test), shuffle=False)

Epoch 1/200
66/66 [==============================] - 117s 2s/step - loss: 0.9987 - accuracy: 0.5305 - jaccard_coef: 0.2645 - val_loss: 0.9887 - val_accuracy: 0.5742 - val_jaccard_coef: 0.3473
Epoch 2/200
66/66 [==============================] - 110s 2s/step - loss: 0.9687 - accuracy: 0.6683 - jaccard_coef: 0.4093 - val_loss: 0.9639 - val_accuracy: 0.7141 - val_jaccard_coef: 0.4302
Epoch 3/200
66/66 [==============================] - 109s 2s/step - loss: 0.9500 - accuracy: 0.7345 - jaccard_coef: 0.4942 - val_loss: 0.9544 - val_accuracy: 0.7222 - val_jaccard_coef: 0.4694
Epoch 4/200
66/66 [==============================] - 108s 2s/step - loss: 0.9439 - accuracy: 0.7550 - jaccard_coef: 0.5264 - val_loss: 0.9544 - val_accuracy: 0.7225 - val_jaccard_coef: 0.4445
Epoch 5/200
66/66 [==============================] - 109s 2s/step - loss: 0.9381 - accuracy: 0.7755 - jaccard_coef: 0.5567 - val_loss: 0.9472 - val_accuracy: 0.7481 - val_jaccard_coef: 0.5189
Epoch 6/200
66/66 [=====================